In [ ]:
import sqlite3
import pandas as pd

In [ ]:
batch_name = "error_injection_timing_test"

con = sqlite3.connect(
        f"/home/jupyter/tf/models/batch_run/{batch_name}/batch_results.sqlite"
    )

query = """
SELECT 
    code_name, 
    epoch, 
    timetick, 
    y,
    testset,
    item, 
    acc, 
    sse
FROM strain
"""


batch_config = pd.read_sql_query("SELECT * FROM batch_config", con)
df = pd.read_sql_query(query, con)
df = df.merge(
        batch_config[["code_name", "inject_error_ticks"]], "left", on="code_name"
    )
df[["testset_name", "freq", "cons", "img"]] = df.testset.str.split("_", expand=True)


In [ ]:
cur = con.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

In [ ]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()
dataset = client.create_dataset(batch_name)

In [ ]:
tables = ('grain', 'taraban', 'strain', 'batch_config')

for t in tables:
    df = pd.read_sql_query(f"SELECT * FROM {t}", con)
    table_ref = dataset.table(t)
    job = client.load_table_from_dataframe(df, table_ref)
    job.result()
    print("Loaded dataframe to {}".format(table_ref.path))
    